In [ ]:
%matplotlib notebook
%precision 3

from IPython.display import set_matplotlib_formats
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import matplotlib
import math
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [9.5, 5]
import pandas

In [ ]:
# C. D. Keeling, S. C. Piper, R. B. Bacastow, M. Wahlen, T. P. Whorf, M. Heimann, and H. A. Meijer, Exchanges of atmospheric CO2 and 13CO2 with the terrestrial biosphere and oceans from 1978 to 2000. I. Global aspects, SIO Reference Series, No. 01-06, Scripps Institution of Oceanography, San Diego, 88 pages, 2001.

In [ ]:
# source: https://scrippsco2.ucsd.edu/data/atmospheric_co2/mlo.html
co2s=pandas.read_csv("https://scrippsco2.ucsd.edu/assets/data/atmospheric/stations/in_situ_co2/weekly/weekly_in_situ_co2_mlo.csv", skiprows=44, header=0,names=["date", "co2ppm"])
co2s["timestamp"]=pandas.to_datetime(co2s["date"],infer_datetime_format=True)
co2s["doy"]=co2s.timestamp.dt.day_of_year
co2s["year"]=co2s.timestamp.dt.year
co2s.set_index("timestamp", inplace=True)


In [ ]:
co2s

In [ ]:
plt.figure()
plt.plot(co2s.co2ppm)
plt.plot(co2s.co2ppm.rolling(52, center=True).mean())
plt.title('Mauna Loa Observatory CO2 PPM')
plt.ylabel("ppm")
plt.grid()

In [ ]:
plt.figure()
absol=np.abs(co2s.co2ppm.rolling(52, center=True).mean() - co2s.co2ppm)
plt.plot(absol)
plt.plot(absol.rolling("365d").mean())
plt.title("Absolute deviation of Mauna Loa Observatory CO2 PPM from annualized data")
plt.ylabel("abs ppm")
plt.grid()

In [ ]:
plt.figure()
co2s["devia"] = co2s.co2ppm- co2s.co2ppm.rolling(52, center=True).mean()
plt.plot(co2s.devia)
plt.title("Deviation of Mauna Loa Observatory CO2 PPM from annualized data")
plt.ylabel("ppm")

plt.grid()

In [ ]:
clean=co2s.dropna()
clean

In [ ]:
part1=clean["2000-01-01":]
coeff1 = np.polyfit(part1.doy, part1.devia, 7)
part2=clean[:"2000-01-01"]
coeff2 = np.polyfit(part2.doy, part2.devia, 7)

days=np.linspace(0, 365, 52)

In [ ]:
plt.figure()
plt.scatter(part1.doy, part1.devia, s=0.1)
plt.scatter(part2.doy, part2.devia, s=0.1)
plt.plot(days, np.polyval(coeff1, days), label="Deviation <2000")
plt.plot(days, np.polyval(coeff2, days), label="Deviation >=2000")
plt.grid()
plt.legend()
plt.title("Deviation of annualized Mauna Loa Observatory CO2 PPM per day of year")

In [ ]:
plt.figure()
trend=co2s.co2ppm - np.polyval(coeff2, co2s.doy)
plt.plot(trend, label="With annual swing removed")
plt.plot(co2s.co2ppm, alpha=0.2, color='blue', label="Unadjusted")
plt.title("Mauna Loa Observatory CO2 PPM with/without annual swing removed")
plt.ylabel("ppm")
plt.legend()
plt.grid()

In [ ]:
plt.figure()
plt.plot(52*trend["1989-01-01":].diff().rolling(52).mean(), label="Lagging yearly average")
plt.plot(52*trend["1980-01-01":].diff().rolling(520).mean(), label="Lagging 10 year average")

plt.grid()
plt.ylabel("PPM/year")
plt.axhline(2.4, color='red', xmin=0.83, xmax=0.97, label='2.4ppm/year')
plt.axhline(1.935, color='red', xmin=0.41, xmax=0.58, label='1.935ppm/year')

plt.axhline(1.5, color='red',xmin=0.05, xmax=0.35, label='1.5ppm/year')

plt.title("Yearly rate of increase of Mauna Loa Observatory CO2 PPM compensated for annual variation")
plt.legend()

In [ ]:
plt.figure()
yearlies=co2s.groupby(["year"]).co2ppm.mean().diff().dropna()
res=yearlies.reset_index()
res=res[(res.year!=2023) & (res.year > 1990)]
plt.ylim(0,4)
coeffg=np.polyfit(res.year, res.co2ppm, 2)
plt.plot(res.year, np.polyval(coeffg,res.year))
plt.plot(res.year, res.co2ppm, '-+')
plt.grid()
plt.title("Yearly change in Mauna Loa Observatory CO$_2$ concentration")